In [2]:
#import necessary libraries
import pandas as pd
import numpy as np
import json
import requests
from pandas.io.json import json_normalize
import urllib.request
!pip install beautifulsoup4
!pip install lxml
from bs4 import BeautifulSoup
!conda install -c conda-forge geopy --yes
from geopy.geocoders import Nominatim


Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [3]:
#define url
url="https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M"
#request url       
html=urllib.request.urlopen(url)

#parse html with beautiful soup
soup = BeautifulSoup(html, "html.parser")
#from inspecting the source code, the data we are interested in is under a header called table, class=sortable, so we find that and assign it to 'table'
table=soup.find('table', class_='wikitable sortable')
# create a dataframe using pandas read_html
df = pd.read_html(str(table))[0]
df.head()


,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"


In [4]:
#cleaning up data and ignore cells with a borough that is Not assigned.
notassigned = df[df['Borough'] == 'Not assigned' ].index

df=df.drop(notassigned).reset_index(drop=True) 
df.head()


,Postal Code,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [5]:
#combine neighbourhoods with same postal code
df_grouped = df.groupby(['Postal Code', 'Borough'])['Neighbourhood'].apply(', '.join).reset_index()
df_grouped.head()



,Postal Code,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [6]:
# Searching for Not assigned neighbourhood reveals there are none
df_grouped.loc[df['Neighbourhood']=='Not assigned']
    

,Postal Code,Borough,Neighbourhood


In [7]:
# Shape of Dataframe
df_grouped.shape

(103, 3)

In [8]:
#install and import geocode code 
!pip install geocoder
import geocoder # import geocoder



In [9]:
# initialize your variable to None
lat_lng_coords = None

#create empty list for latitude and longitude
Latitude=[]
Longitude=[]

#create for loop to loop through all postal codes
#for postal_code, neighborhood in zip(df['Postal Code'], df['Neighborhood']):
    # loop until you get the coordinates
    #postal_code=postal_code
    #neighborhood=neighborhood
    
    #while(lat_lng_coords is None):
        #g = geocoder.google('{}, neighborhood'.format(postal_code))
        #lat_lng_coords = g.latlng
        
        #latitude = lat_lng_coords[0]
        #longitude = lat_lng_coords[1]
        #Latitude.append(latitude)
        #Longitude.append(longitude)
      
#create new columns in df:
#df['Latidude']=Latitude
#df['Longitude']=Longitude
#df.head()


In [9]:
# The geocoder does not appear to work, so I left it for reference of how I think it would work

In [10]:
# I'll import the csv file from the url provided in the assignment

!wget -O Geospacial_Coordinates.csv https://cocl.us/Geospatial_data
df1=pd.read_csv('Geospacial_Coordinates.csv')
df1.head()

--2020-08-24 16:18:13--  https://cocl.us/Geospatial_data
Resolving cocl.us (cocl.us)... 169.55.161.7
Connecting to cocl.us (cocl.us)|169.55.161.7|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-08-24 16:18:14--  https://ibm.box.com/shared/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Resolving ibm.box.com (ibm.box.com)... 107.152.27.197
Connecting to ibm.box.com (ibm.box.com)|107.152.27.197|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: /public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-08-24 16:18:14--  https://ibm.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv
Reusing existing connection to ibm.box.com:443.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://ibm.ent.box.com/public/static/9afzr83pps4pwf2smjjcf1y5mvgb18rr.csv [following]
--2020-08-24 16:1

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [11]:
#now we merge df and df1 to get the new dataframe, named df_new
df_new=pd.merge(df, df1, on="Postal Code")
df_new.head()

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494


In [12]:
# First we create an instance of the geocoder named toronto_explorer
address = 'Toronto'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


Create a map of Toronto with neighborhoods superimposed on top. For That I need to install Folium

In [13]:
!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library



Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [14]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, borough, neighborhood in zip(df_new['Latitude'], df_new['Longitude'], df_new['Borough'], df_new['Neighbourhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

Now I define my Foursquare Credentials

In [15]:
 # Foursquare API version


I will now explore the first 50 venues in the Toronto  within a radius of 500 meters

In [16]:
LIMIT = 50  #limit of number of venues returned 

radius = 500 # define radius



 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL


'https://api.foursquare.com/v2/venues/explore?&client_id=RYQE1EXFHEL3K3YNEVIJTHVPH3SRSG1ILXRDGUE2O4S2U00S&client_secret=B4HVCK5SUARZZEGSPMRS4WVWWD3CYV2URNYNFSPJDDOJ4IMS&v=20180605&ll=43.6534817,-79.3839347&radius=500&limit=50'

Send GET REQUEST AND EXAMINE RESULTS

In [17]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f43e7d23c86252883b572a3'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Bay Street Corridor',
  'headerFullLocation': 'Bay Street Corridor, Toronto',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 85,
  'suggestedBounds': {'ne': {'lat': 43.6579817045, 'lng': -79.37772678059432},
   'sw': {'lat': 43.6489816955, 'lng': -79.39014261940568}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '5227bb01498e17bf485e6202',
       'name': 'Downtown Toronto',
       'location': {'lat': 43.65323167517444,
        'lng': -79.38529600606677,
        'labeledLatLngs': [{'label': 'display',
          'lat': 43.65323167517444,
          'lng'

Now I will use the function defined in a previous class to get the venue type and then turn the results into a dataframe:
      

In [18]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

Clean Data and form a Dataframe

In [19]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,Downtown Toronto,Neighborhood,43.653232,-79.385296
1,Nathan Phillips Square,Plaza,43.652270,-79.383516
2,Japango,Sushi Restaurant,43.655268,-79.385165
3,Eggspectation Bell Trinity Square,Breakfast Spot,43.653144,-79.381980
4,Chatime 日出茶太,Bubble Tea Shop,43.655542,-79.384684


The first result on the list is not actually a venue, so I am going to drop it from the frame


In [20]:
nearby_venues=nearby_venues.drop([0])

Number of Places returned

In [21]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

49 venues were returned by Foursquare.


Now we find the top 5 most visited venues

In [22]:
nearby_ven_grouped=nearby_venues.groupby(['categories']).mean().reset_index()
nearby_ven_grouped.head(5)

,categories,lat,lng
0,American Restaurant,43.651569,-79.379266
1,Art Museum,43.654396,-79.386500
2,Bank,43.653937,-79.379722
3,Bar,43.650557,-79.381956
4,Bookstore,43.653515,-79.380696


In [23]:
top5=""
for cat in nearby_ven_grouped.iloc[0:5,0] :
    top5=top5 + '\n' + cat
print('The most visited venues were: {}'.format(top5))

The most visited venues were: 
American Restaurant
Art Museum
Bank
Bar
Bookstore
